# LEED Diffraction Pattern Recognition for Simulations of Material Property 

-------------------------------------------------------------------------------

## 1. Designing a Neural Network for LEED Images

### 1.1 Define Architecture of Network
![Overview](overview_sheme.png)

<br>

```python
class BravaisLatticeRecognitionNet(nn.Module):
    def __init__(self, num_points, vocab_size=5, hidden_dim=IMAGE_DIM*IMAGE_DIM, num_layers=3):
        super(BravaisLatticeRecognitionNet, self).__init__()
        self.hidden_layers = nn.ModuleList()
        self.input_dim = num_points
        for i in range(num_layers):
            in_dim = self.input_dim if i == 0 else hidden_dim
            self.hidden_layers.append(nn.Linear(in_dim, hidden_dim))
        self.output_layer = nn.Linear(hidden_dim, vocab_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        return self.output_layer(x)
```

<br>



### 1.2 Choose Loss - Function

```python
loss_fn = nn.CrossEntropyLoss() # Set Cross Entropy as Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"]) # Choose optimizer and set learning rate
```

### 1.3 Set up Hyperparameters: ###

In [28]:
# Training settings
BATCH_SIZE = 4 # Trial and Error
NUM_ITERATIONS = 500 # Trial and Error
LEARNING_RATE = 5e-3 # Trial and Error

# Model architecture
HIDDEN_DIM = 256 # Trial and Error / Informed Guess
NUM_LAYERS = 4 # Trial and Error
ACTIVATION_FUNC = "ReLU" # Informed Guess

# Data settings
IMAGE_DIM = 128 # size of the input image
VOCAB_SIZE = 5  # number of Bravais lattice types
K_MAX_PEAKS = 32  # number of peaks to extract



-------------------------------------------------------------------------------

## 2. Physical Background

### 2.1 Assumptions due to LEED Diffraction:
- Elastic scattering of the electrons
- Low penetration depth of electrons (first two lattice layers)
- Laue Condition 


-------------------------------------------------------------------------------

### 2.2 Laue Condition:

The 2D Laue condition for diffraction is given by:

$$
\vec{k}_{\text{out}} - \vec{k}_{\text{in}} = \vec{G}_{hk} 
$$

Where $\vec{k}_{\text{in}}$ is the incident wavevector,  
$\vec{k}_{\text{out}}$ is the diffracted wavevector,  
and $\vec{G}_{hk} $ is a reciprocal lattice vector defined as:
$$
\vec{G}_{hk} = h \vec{b}_1 + k \vec{b}_2
$$

with \( h, k \) (Miller-Indizes) and the reciprocal base vectors $\vec{b}_i$.

-------------------------------------------------------------------------------

### 2.3 Relation reciprocal and real lattice vectors:

$$
\vec{a}_1 = (a_{1x}, a_{1y}, 0), \quad 
\vec{a}_2 = (a_{2x}, a_{2y}, 0), \quad 
\vec{a}_3 = \vec{e}_z = (0, 0, 1)
$$

The Volume of the real basis vectors:

$$
V = \vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)
$$

Reciprocal lattice vectors:

$$
\vec{b}_1 = 2\pi \frac{\vec{a}_2 \times \vec{a}_3}{V}, \quad
\vec{b}_2 = 2\pi \frac{\vec{a}_3 \times \vec{a}_1}{V}, \quad
\vec{b}_3 = 2\pi \frac{\vec{a}_1 \times \vec{a}_2}{V}
$$

Orthogonality:

$$
\vec{a}_i \cdot \vec{b}_j = 2\pi \delta_{ij}
$$

-------------------------------------------------------------------------------


### 2.4 Intensity of LEED Diffraction Maxima

In the kinematic approximation, the intensity of a diffraction maximum in LEED is given by:

$$
I(\vec{G}) \propto \left| \sum_{j=1}^{N} f_j \, e^{i \vec{G} \cdot \vec{r}_j} \right|^2
$$

This equation describes how the arrangement of atoms in the 2D surface unit cell affects the scattered intensity.

**Explanation of terms:**

- $I(\vec{G})$ — Intensity of the diffracted electron beam in direction $\vec{G}$
- $f_j$ — Scattering amplitude (form factor) of atom $j$, depends on element type and electron energy
- $\vec{r}_j$ — Position of atom $j$ within the 2D unit cell
- $N$ — Number of atoms in the unit cell

The term inside the modulus is called the **structure factor**:

> *Note:* Here in this Project i just consider a **single atom per unit cell** and only **single elastic scattering**. So in that case i make **not use** of **first principle methods**.


-------------------------


## 3. Generate Training Data

### 3.1 Define 5 different real bravais lattice vectors:

```python
# --- Bravais Gitterdefinitionen ---
BRAVAIS_TYPES = {
    "tp": (np.array([1, 0], dtype=np.float32), np.array([0, 1], dtype=np.float32)),
    "op": (np.array([1, 0], dtype=np.float32), np.array([0, 2], dtype=np.float32)),
    "oc": (np.array([0.5, 0.5], dtype=np.float32), np.array([0.5, -0.5], dtype=np.float32)),
    "hp": (np.array([1, 0], dtype=np.float32), np.array([0.5, np.sqrt(3)/2], dtype=np.float32)),
    "mp": (np.array([1, 0], dtype=np.float32), np.array([0.3420, 0.9397], dtype=np.float32))
}

### 3.2 Calculate the reciprocal lattice vectors:
```python
# --- Caluclate reciprocal lattice ---
def reciprocal_lattice_2d(a1, a2, a):
    a1_3d = np.array([a1[0], a1[1], 0]) * a
    a2_3d = np.array([a2[0], a2[1], 0]) * a
    area = np.cross(a1_3d, a2_3d)[2]
    b1 = 2 * np.pi * np.cross([0, 0, 1], a2_3d)[:2] / area
    b2 = 2 * np.pi * np.cross(a1_3d, [0, 0, 1])[:2] / area
    return b1.astype(np.float32), b2.astype(np.float32)

### 3.3 Generate reciprocal lattice points:
```python
# --- Generierung der Gitterpunkte ---
def generate_lattice_points(b1, b2, N):
    lattice_points = []
    for i in range(-N, N+1):
        for j in range(-N, N+1):
            point = i * b1 + j * b2
            lattice_points.append(point)
    return np.array(lattice_points).T.astype(np.float32)  # shape: (2, (2N+1)^2)


### 3.4 Place 2D-Gaussians over the reciprocal lattice points:
```python
# --- Transform infinite sharp peaks to 2D Gaussian for leed image generation ---
def gaussian_2d(x, y, means, sigma_x=0.01, sigma_y=0.01):
    X, Y = np.meshgrid(x, y)
    means = np.array(means.tolist())
    print("means:", means)
    Z = np.zeros(X.shape)
    for i in range(means.shape[0]):
        a = 1 / (2 * np.pi * sigma_x * sigma_y)
        x0 = means[i, 0]
        y0 = means[i, 1]
        b = -((X - x0) ** 2 / (2 * sigma_x ** 2) + (Y - y0) ** 2 / (2 *     sigma_y ** 2))
        Z += a * np.exp(b)   
    return Z

### 3.5 Generate LEED - Images 128x128 for different parameters:
```python
a_list = [1, 3, 5, 2] # Lattice constant
sigma_list = [0.01, 0.03, 0.015, 0.02] # Noise of Gaussian
n_points_list = [32, 96, 128, 256, 64, 48] # Number of maxima peaks
```

#### Resulting Dataset:
- Train set: 384 Images (80 %)
- Test set: 96 Images (20 %)

-------------------------------------------------------

### 3.7 Example Images: ###
![test_sample1](test_sample1.png)
![test_sample2](test_sample2.png)
![test_sample3](test_sample3.png)
![test_sample4](test_sample4.png)
![test_sample5](test_sample5.png)

---------------------------------

## 4. Train Neural Network 

```python
if hasattr(tqdm, '_instances'): tqdm._instances.clear()
for iter in tqdm(range(params["num_training_iterations"])):
    train_loader, _, _ = get_train_data_from_file(batch_size=BATCH_SIZE) # Get Train Set
    train_loader_peaks_xy, train_loader_labels = translate_batch_image_to_peak_coordinates_batch(train_loader) # Identify Peaks returns x, y Component
    
    model.train() # Set train mode (random zeros)
    optimizer.zero_grad() # Calculate Gradients
    y_hat = model(train_loader_peaks_xy) # predict
    loss = loss_fn(y_hat, train_loader_labels) # Evaluate Loss
    loss.backward() # Backpropagation
    optimizer.step() # Optimize

    # Log metrics to Comet
    experiment.log_metric("loss", loss.item(), step=iter)
    history.append(loss.item())
    if iter % 100 == 0:
        torch.save(model.state_dict(), checkpoint_prefix)

torch.save(model.state_dict(), checkpoint_prefix)
experiment.flush()
experiment.end()
```
--------------------

## 5. Results: 

### 5.1 Loss vs Step:
![Loss vs Step](lossvsstep.jpeg)
--------------------
### 5.2 Best found Hyperparameters:
```python
# Training settings
BATCH_SIZE = 6
NUM_ITERATIONS = 500
LEARNING_RATE = 5e-3

# Model architecture
HIDDEN_DIM = 312
NUM_LAYERS = 1
ACTIVATION_FUNC = "ReLU"
```
--------------------
### 5.3 Confusion Matrix:
![ConfusionMatrix](confusion_matrix.png)
--------------------

#### Accuracy: 98.96 %
----------------
---------------
